In [2]:
import tensorflow as tf
import keras
from keras import backend as K
from keras.datasets import mnist
from keras.layers import BatchNormalization, Flatten, Dense
from keras.layers import Input, Conv2D, Convolution2D
from keras.layers import Activation, concatenate, Dropout
from keras.layers import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D

from keras.models import Sequential, Model
from keras.utils import np_utils, to_categorical
from keras.applications.imagenet_utils import _obtain_input_shape

from tuner import utils
from tuner.tune import fit, fit_generator

from hyperas import optim
from hyperopt import Trials, STATUS_OK, tpe
from hyperas.distributions import choice, uniform, conditional

In [3]:
train_dir = 'dataset/brain/train'
validation = 'dataset/brain/validation'
df_train = utils.df_fromdir(train_dir)
df_test = utils.df_fromdir(train_dir)

x_train, y_train = utils.load_fromdf(df_train)
x_test, y_test = utils.load_fromdf(df_test)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [4]:
import Augmentor 

In [6]:
#@compute_loss(epochs=200)
def tuing_aug(n_out, input_shape=(96, 96, 3)):
    p = Augmentor.Pipeline()
    p.flip_left_right(probability=0.5)
    p.rondom_croping(probability=1, percentage_area=0.8)
    p.resize(probability=1, width=96, height=96)
    p.status()
    g = p.keras_generator_from_array(x_train, y_train, batch_size=args.bs)
    g = ( (x/255., y) for (x,y) in g)

    
    inputs = Input(shape=input_shape)
    x = inputs
    x = Conv2D(32, (3, 3))(x)
    x = Conv2D(32, (3, 3))(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Conv2D(64, (3, 3))(x)
    x = Conv2D(64, (3, 3))(x)
    x = Activation('relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Dropout(0.25)(x)
    x = Flatten()(x)
    x = Dense(512)(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(n_out)(x)
    x = Activation('softmax')(x)
    model = Model(inputs=inputs, outputs=x)
    
    model.fit_generator(g, validation_data=(x_test, y_test),
                        batch_size=batch_size,
                        epochs=epochs,
                        steps_per_epoch=steps_per_epoch,
                        verbose=2,
                       )
    score, acc = model.evaluate(x_test, y_test, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model':model}

In [30]:
op = p.operations[0]
dir(op)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'perform_operation',
 'probability',
 'top_bottom_left_right']

In [23]:
dir(p)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_execute',
 '_execute_with_array',
 '_legal_filters',
 '_populate',
 '_probability_error_text',
 '_threshold_error_text',
 '_valid_formats',
 'add_further_directory',
 'add_operation',
 'augmentor_images',
 'black_and_white',
 'categorical_labels',
 'class_labels',
 'crop_by_size',
 'crop_centre',
 'crop_random',
 'distinct_dimensions',
 'distinct_formats',
 'flip_left_right',
 'flip_random',
 'flip_top_bottom',
 'gaussian_distortion',
 'greyscale',
 'histogram_equalisation',
 'image_counter',
 'image_generator',
 'invert',
 'keras_generator',
 'keras_generator_from_array',
 'operations',
 'random_distortion',
 'random_e

In [32]:
p.status()

Operations: 3
	0: Flip (probability=0.5 top_bottom_left_right=LEFT_RIGHT )
	1: CropPercentage (probability=1 percentage_area=0.8 centre=False randomise_percentage_area=False )
	2: Resize (probability=1 width=96 height=96 resample_filter=BICUBIC )
Images: 0
Classes: 0

You can remove operations using the appropriate index and the remove_operation(index) function.


In [40]:
p3 = Augmentor.Pipeline()

In [41]:
for i, op in enumerate(p.operations):
    print(i, op, op.__dict__.items())
    
    p2.add_operation(op)
    p3.add_operation(op)

0 Flip dict_items([('probability', 0.5), ('top_bottom_left_right', 'LEFT_RIGHT')])
1 CropPercentage dict_items([('probability', 1), ('percentage_area', 0.8), ('centre', False), ('randomise_percentage_area', False)])
2 Resize dict_items([('probability', 1), ('width', 96), ('height', 96), ('resample_filter', 'BICUBIC')])


In [43]:
with open('tmp.txt', '') as f:
    for i, op in enumerate(p.operations):
        f.write(op)

TypeError: write() argument must be str, not Flip

In [12]:
batch_size = 32
p = Augmentor.Pipeline()
p.flip_left_right(probability=0.5)
p.crop_random(probability=1, percentage_area=0.8)
p.resize(probability=1, width=96, height=96)
p.status()
g = p.keras_generator_from_array(x_train, y_train, batch_size=batch_size)
g = ( (x/255., y) for (x,y) in g)

Operations: 3
	0: Flip (probability=0.5 top_bottom_left_right=LEFT_RIGHT )
	1: CropPercentage (probability=1 percentage_area=0.8 centre=False randomise_percentage_area=False )
	2: Resize (probability=1 width=96 height=96 resample_filter=BICUBIC )
Images: 0
Classes: 0

You can remove operations using the appropriate index and the remove_operation(index) function.


In [13]:
next(g)

(array([[[[ 0.        ,  0.        ,  0.        ],
          [ 0.03529412,  0.03529412,  0.03529412],
          [ 0.1254902 ,  0.1254902 ,  0.1254902 ],
          ..., 
          [ 0.        ,  0.        ,  0.        ],
          [ 0.        ,  0.        ,  0.        ],
          [ 0.        ,  0.        ,  0.        ]],
 
         [[ 0.0627451 ,  0.0627451 ,  0.0627451 ],
          [ 0.16862745,  0.16862745,  0.16862745],
          [ 0.32156863,  0.32156863,  0.32156863],
          ..., 
          [ 0.        ,  0.        ,  0.        ],
          [ 0.        ,  0.        ,  0.        ],
          [ 0.        ,  0.        ,  0.        ]],
 
         [[ 0.21960784,  0.21960784,  0.21960784],
          [ 0.36862745,  0.36862745,  0.36862745],
          [ 0.48627451,  0.48627451,  0.48627451],
          ..., 
          [ 0.        ,  0.        ,  0.        ],
          [ 0.        ,  0.        ,  0.        ],
          [ 0.        ,  0.        ,  0.        ]],
 
         ..., 
         [

In [10]:
lenet(x_train, y_train, x_test, y_test)

build model
Train on 226 samples, validate on 226 samples
Epoch 1/200
 - 1s - loss: 10.6597 - acc: 0.3319 - val_loss: 10.8405 - val_acc: 0.3274
Epoch 2/200
 - 0s - loss: 10.8405 - acc: 0.3274 - val_loss: 10.8405 - val_acc: 0.3274
Epoch 3/200
 - 0s - loss: 10.8405 - acc: 0.3274 - val_loss: 10.8405 - val_acc: 0.3274
Epoch 4/200
 - 0s - loss: 10.8405 - acc: 0.3274 - val_loss: 10.8405 - val_acc: 0.3274
Epoch 5/200
 - 0s - loss: 10.8405 - acc: 0.3274 - val_loss: 10.8405 - val_acc: 0.3274
Epoch 6/200
 - 0s - loss: 10.8405 - acc: 0.3274 - val_loss: 10.8405 - val_acc: 0.3274
Epoch 7/200
 - 0s - loss: 10.8405 - acc: 0.3274 - val_loss: 10.8405 - val_acc: 0.3274
Epoch 8/200
 - 0s - loss: 10.8405 - acc: 0.3274 - val_loss: 10.8405 - val_acc: 0.3274
Epoch 9/200
 - 0s - loss: 10.8405 - acc: 0.3274 - val_loss: 10.8405 - val_acc: 0.3274
Epoch 10/200
 - 0s - loss: 10.8405 - acc: 0.3274 - val_loss: 10.8405 - val_acc: 0.3274
Epoch 11/200
 - 0s - loss: 10.8405 - acc: 0.3274 - val_loss: 10.8405 - val_acc: 0

KeyboardInterrupt: 

In [24]:
p2 = Augmentor.Pipeline()

In [25]:
p2.operations

[]

In [26]:
dir(p2)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_execute',
 '_execute_with_array',
 '_legal_filters',
 '_populate',
 '_probability_error_text',
 '_threshold_error_text',
 '_valid_formats',
 'add_further_directory',
 'add_operation',
 'augmentor_images',
 'black_and_white',
 'categorical_labels',
 'class_labels',
 'crop_by_size',
 'crop_centre',
 'crop_random',
 'distinct_dimensions',
 'distinct_formats',
 'flip_left_right',
 'flip_random',
 'flip_top_bottom',
 'gaussian_distortion',
 'greyscale',
 'histogram_equalisation',
 'image_counter',
 'image_generator',
 'invert',
 'keras_generator',
 'keras_generator_from_array',
 'operations',
 'random_distortion',
 'random_e

In [27]:
[ p2.add_operation(op) for op in p.operations]

[None, None, None]